In [1]:
import sqlite3
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
conn.enable_load_extension(True)

for (val,) in cur.execute('pragma compile_options'): 
    print (val)

COMPILER=msvc-1916
ENABLE_COLUMN_METADATA
ENABLE_DBSTAT_VTAB
ENABLE_FTS3
ENABLE_FTS3_TOKENIZER
ENABLE_FTS4
ENABLE_FTS5
ENABLE_GEOPOLY
ENABLE_JSON1
ENABLE_RTREE
ENABLE_UNLOCK_NOTIFY
MAX_DEFAULT_PAGE_SIZE=32768
MAX_VARIABLE_NUMBER=250000
SECURE_DELETE
THREADSAFE=1


https://komorandocs.readthedocs.io/ko/latest/pykomoran/tutorial.html

In [3]:
from PyKomoran import *
komoran = Komoran(DEFAULT_MODEL['LIGHT'])
komoran.set_user_dic('dic/user.dic')
komoran.get_nouns("김문근은 천재인가요")

['김문근', '천재']

In [23]:
from peewee import *
from playhouse.sqlite_ext import *
import datetime
from PyKomoran import *
komoran = Komoran(DEFAULT_MODEL['LIGHT'])
komoran.set_user_dic('dic/user.dic')

# pragmas = [
#     ('journal_mode', 'wal'),
#     ('cache_size', -1024 * 32)]
# db = SqliteExtDatabase('blog.db', pragmas=pragmas)
db = SqliteExtDatabase('treememo.db')


class Document(Model):
    title = TextField()
    create_at = DateTimeField(default=datetime.datetime.now)
    modify_at = DateTimeField(default=datetime.datetime.now)

    class Meta:
        database = db
    
class DocumentEntry(Model):
    document_id = ForeignKeyField(Document, backref='map_document')
    entry_id = ForeignKeyField(Entry, backref='map_entry')
    parent_id = ForeignKeyField('self', null=True, backref='map_parent')
    prev_id = ForeignKeyField('self', null=True, backref='map_prev')

    class Meta:
        database = db
   
class Entry(Model):
    content = TextField()
    create_at = DateTimeField(default=datetime.datetime.now)
    modify_at = DateTimeField(default=datetime.datetime.now)

    class Meta:
        database = db
        
class FTSEntry(FTSModel):
    content = TextField()

    class Meta:
        database = db        
        
class Tag(Model):
    title = TextField()
    create_at = DateTimeField(default=datetime.datetime.now)
    modify_at = DateTimeField(default=datetime.datetime.now)

    class Meta:
        database = db
        
class TagMap(Model):
    tag_id = ForeignKeyField(Tag, backref='map_tag')
    entry_id = ForeignKeyField(Entry, backref='map_entry')

    class Meta:
        database = db
    
def add_content(content):
    entry = Entry.create(content=content)
    FTSEntry.create(docid=entry.id, content='\n'.join(komoran.get_nouns(content)))
    return entry
    

def get_tag(title):
    try:
        tag = Tag.get(Tag.title == title)
        print('exist - ', title, tag.title, tag.id)
    except:    
        tag = Tag.create(title=title)
        print('new - ', title, tag.title, tag.id)
    return tag
    
def add_tag(docid, title):
    tag = get_tag(title)
    print(tag)
    TagMap.create(tag_id=tag.id, entry_id=docid)
    
    
def find_entry(content):
    return [entry for entry in Entry.select().join(FTSEntry, on=(Entry.id == FTSEntry.docid)).where(FTSEntry.match(content)).order_by(FTSEntry.rank()).dicts()]

def find_tag(title):
    return [tag for tag in Tag.select().where(Tag.title.startswith(title)).dicts()]

def find_tag_entry(tag_id):
    return [entry for entry in Entry.select().join(TagMap, on=TagMap.entry_id).where(TagMap.tag_id == tag_id).dicts()]

def add_document(title):
    return Document.create(title=title)

def add_document_entry(document_id, entry_id, parent_id, prev_id):
    return DocumentEntry.create(document_id=document_id, entry_id=entry_id, parent_id=parent_id, prev_id=prev_id)
    
def get_document_entries(document_id):
    return [document_entry for document_entry in DocumentEntry.select().where(DocumentEntry.document_id == document_id).dicts()]
    
db.create_tables([Entry, FTSEntry, Tag, TagMap, Document, DocumentEntry])    

In [5]:
doc = add_content('김문근')
doc = add_content('김문근 천재')
doc = add_content('김문근 천재 인가요?')

In [15]:
find_entry('천재')

[{'id': 2,
  'content': '김문근 천재',
  'create_at': datetime.datetime(2020, 12, 6, 11, 43, 41, 709466),
  'modify_at': datetime.datetime(2020, 12, 6, 11, 43, 41, 709466)},
 {'id': 3,
  'content': '김문근 천재 인가요?',
  'create_at': datetime.datetime(2020, 12, 6, 11, 43, 41, 718445),
  'modify_at': datetime.datetime(2020, 12, 6, 11, 43, 41, 718445)}]

In [16]:
find_tag('인')

[{'id': 1,
  'title': '인물',
  'create_at': datetime.datetime(2020, 12, 6, 11, 44, 5, 194307),
  'modify_at': datetime.datetime(2020, 12, 6, 11, 44, 5, 194307)}]

In [7]:
add_tag(1, '인물')

new -  인물 인물 1
1


In [123]:
add_tag(1, '인사')
add_tag(2, '인사')

new -  인사 인사 2
2
exist -  인사 인사 2
2


In [8]:
find_tag_entry(1)

[{'id': 1,
  'content': '김문근',
  'create_at': datetime.datetime(2020, 12, 6, 11, 43, 41, 688384),
  'modify_at': datetime.datetime(2020, 12, 6, 11, 43, 41, 688384)}]

In [21]:
add_document('연습')

<Document: 1>

In [22]:
add_document_entry(1, 1, None, None)

<DocumentEntry: 1>

In [24]:
get_document_entries(1)

[{'id': 1,
  'document_id': 1,
  'entry_id': 1,
  'parent_id': None,
  'prev_id': None}]